# PointNet

This notebook allows to train a model for classification of pointclouds in 10 classes with a 82% accuracy.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

!pip install path.py;
from path import Path
root_dir = Path("/content/gdrive/My Drive/PointNet3D")
import sys
sys.path.append(root_dir)

Mounted at /content/gdrive


In [0]:
import model
import dataset3D
import utils
import torch
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split

## Data Load

In [0]:
data_dir = Path("/datasets/ModelNet10txt/ModelNet10/ModelNet10/")

In [0]:
classes = {'bathtub': 0, 'bed': 1, 'chair': 2, 'desk': 3, 'dresser': 4, 'monitor': 5, 'night_stand': 6, 'sofa': 7, 'table': 8, 'toilet': 9}

In [16]:
dset = dataset3D.Data3D(root_dir + data_dir, classes)
train_num = int(len(dset) * 0.95)
val_num = int(len(dset) *0.05)
if int(len(dset)) - train_num -  val_num >0 :
    train_num = train_num + 1
elif int(len(dset)) - train_num -  val_num < 0:
    train_num = train_num -1
train_dataset, val_dataset = random_split(dset, [train_num, val_num])
val_dataset.valid=True

print('######### Dataset class created #########')
print('Number of images: ', len(dset))
print('Number of categories: ', len(dset.categories))
print('Sample image shape: ', dset[1000]['image'], dset[1000]['category'], end='\n\n')


######### Dataset class created #########
Number of images:  3118
Number of categories:  10
Sample image shape:  [[ 0.23952495 -0.02368225 -0.00872219]
 [-0.08794533 -0.38928795  0.00621358]
 [-0.19961856 -0.017271   -0.19023441]
 ...
 [-0.06298735  0.0590462  -0.08584125]
 [-0.38503152  0.35486215  0.15427132]
 [ 0.45372537  0.04931326  0.2109515 ]] 2



In [0]:
train_loader = DataLoader(dataset=train_dataset, batch_size=32)
val_loader = DataLoader(dataset=val_dataset, batch_size=32)

## Training loop

In [23]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [0]:
pointnet = model.PointNet()

In [0]:
pointnet.to(device);

In [0]:
optimizer = torch.optim.Adam(pointnet.parameters(), lr=0.001)

In [0]:
def pointnetloss(outputs, labels, m3x3, m64x64, alpha = 0.0001):
    criterion = torch.nn.NLLLoss()
    bs=outputs.size(0)
    id3x3 = torch.eye(3, requires_grad=True).repeat(bs,1,1)
    id64x64 = torch.eye(64, requires_grad=True).repeat(bs,1,1)
    if outputs.is_cuda:
        id3x3=id3x3.cuda()
        id64x64=id64x64.cuda()
    diff3x3 = id3x3-torch.bmm(m3x3,m3x3.transpose(1,2))
    diff64x64 = id64x64-torch.bmm(m64x64,m64x64.transpose(1,2))
    return criterion(outputs, labels) + alpha * (torch.norm(diff3x3)+torch.norm(diff64x64)) / float(bs)
        

In [0]:
def train(model, train_loader, val_loader=None,  epochs=15, save=True):
    for epoch in range(epochs): 
        pointnet.train()
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data['image'].to(device), data['category'].to(device)
            optimizer.zero_grad()
            outputs, m3x3, m64x64 = pointnet(inputs.transpose(1,2))

            loss = pointnetloss(outputs, labels, m3x3, m64x64)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 10 == 9:    # print every 10 mini-batches
                    print('[%d, %5d] loss: %.3f' %
                        (epoch + 1, i + 1, running_loss / 10))
                    running_loss = 0.0

        pointnet.eval()
        correct = total = 0

        # validation
        if val_loader:
            with torch.no_grad():
                for data in val_loader:
                    inputs, labels = data['image'].to(device), data['category'].to(device)
                    outputs, __, __ = pointnet(inputs.transpose(1,2))
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
            val_acc = 100 * correct / total
            print('Valid accuracy: %d %%' % val_acc)

        # save the model
        if save:
            torch.save(pointnet.state_dict(), root_dir+"/models/"+str(epoch)+"_"+str(val_acc))

   

In [0]:
train(pointnet, train_loader, val_loader,  save=True)

## Test

In [0]:
pointnet = model.PointNet()
pointnet.load_state_dict(torch.load(root_dir+"/models/"+"10_93.54838709677419"))
pointnet.eval()

In [0]:
batch = next(iter(val_loader))

In [0]:
pred = pointnet(batch['image'].transpose(1,2))

In [65]:
pred_np = np.array(torch.argmax(pred[0],1));
pred_np

array([1, 1, 2, 1, 3, 5, 9, 1, 9, 8, 2, 6, 8, 2, 9, 1, 7, 2, 5, 3, 8, 8,
       1, 3, 0, 5, 9, 2, 5, 8, 4, 2])

In [68]:
pred_np==np.array(batch['category'])

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False,  True])

In [0]:
x,y,z = np.array(batch['image'][2].T)

In [70]:
utils.pcshow(x,y,z)

In [75]:
list(classes.keys())[pred_np[2]]

'chair'